In [1]:
from utils_csa import save_obj, load_obj

import pandas.io.sql as psql
import pandas as pd
import numpy as np

from pytz import timezone
from datetime import datetime, timedelta, date

ORDER_BUY  = 'buy'
ORDER_SELL = 'sell'  

In [2]:
class Alg_order:
    def __init__(self, id_order, base_asset, quote_asset, side, quantity_base, quantity_quote, price, fees, fees_quote_asset):
        self.id_order = id_order
        self.base_asset = base_asset
        self.quote_asset = quote_asset
        self.side = side
        self.quantity_base = quantity_base
        self.quantity_quote = quantity_quote
        self.price = price
        self.fees = fees
        self.fees_quote_asset = fees_quote_asset
        
class Trading_pair:
    def __init__(self, name, base_asset, quote_asset):
        self.name = name
        self.base_asset = base_asset
        self.quote_asset = quote_asset

In [100]:
# Faire une classe mère Trading_api avec héritage
#class Trading_api_fake(Trading_api):   
class Trading_api_fake:    
    def __init__(self):
        self.positions = {}
        self.fees = 0
        self.nb_periods_price_to_buy = 1 # approx because actions will have delay
        self.orders = {}
        
    def __get_price(self, base_asset, quote_asset, key):
        return self.close_prices[base_asset + quote_asset][key + timedelta(hours=self.nb_periods_price_to_buy)]
    
    def init_from_backtesting_strategy(self, init_positions, fees, close_prices):
        self.positions = init_positions
        self.fees = fees
        self.close_prices = close_prices
        
    def get_available_amount_crypto(self, symbol):
        if symbol in self.positions:
            return self.positions[symbol]
        else:
            return 0.0
        
    def __get_from_to(self, base_asset, quote_asset, side):
        from_crypto = quote_asset
        to_crypto = base_asset
        if side == ORDER_SELL:
            from_crypto = base_asset
            to_crypto = quote_asset
        return from_crypto, to_crypto
    
    # market for the moment => to be scheduled with like market minus 0.5% 
    def create_order(self, base_asset, quote_asset, side, quantity_from, key): # ex: USDT, ETH, 1000, BUY        
        # if cryptos exists
        from_crypto = ''
        to_crypto = ''
        if base_asset in self.positions and quote_asset in self.positions:
            from_crypto, to_crypto = self.__get_from_to(base_asset, quote_asset, side)
            
            # calcul fees
            fees = quantity_from * self.fees
            fees_quote_asset = fees
            
            # get price
            price_init = self.__get_price(base_asset, quote_asset, key)
            price = price_init
            
            # switch when Sell order
            if side == ORDER_SELL:
                fees_quote_asset = fees * price
                price = 1 / price
                
            
            # do exchange between cryptos
            self.positions[from_crypto] = self.positions[from_crypto] - quantity_from
            quantity_to = (quantity_from - fees) / price       
            self.positions[to_crypto] = self.positions[to_crypto] + quantity_to
            base_asset_quantity, quote_asset_quantity = self.__get_from_to(quantity_from, quantity_to, side)
            
            # create order already executed (for simulation needs)
            
            order = Alg_order(len(self.orders), base_asset, quote_asset, side, base_asset_quantity, quote_asset_quantity, price_init, fees, fees_quote_asset)
            self.orders[order.id_order] = order            
            return order.id_order
        else:
            raise ValueError('ERROR (create_order):  cryptos doesn''t exists - ' + from_crypto + '/' + to_crypto)
    
    def get_order(self, id_order):
        return self.orders[id_order]
    
    def get_orders(self):
        return self.orders

class Trading_module:
    signals_x_buy = []
    signals_y_buy = []
    
    x_sell = []
    y_sell = []
    
    amount_x = []
    amount_y = []
    
    def __init__(self, trading_api, param_bet_size, param_min_bet_size, trading_pairs, cash_asset, thresholds, trace):
        self.trading_api = trading_api
        self.trading_pairs = trading_pairs
        self.cash_asset = cash_asset
        self.thresholds = thresholds
        self.trace = trace
        
        self.nb_periods_to_hold_position = 24 #1d => TODO : sell must be done if pct_change model touched / check close price
        self.param_bet_size = param_bet_size
        self.param_min_bet_size = param_min_bet_size
        
    def __can_buy(self):   
        return self.trading_api.get_available_amount_crypto(self.cash_asset) >= self.param_min_bet_size
    
    def __what_to_buy(self, current_date, signals):
        what_to_buy = {}
        max_prob = 0
        max_trading_pair = ''
        if self.__can_buy():
            for trading_pair, trading_pair_probs in signals.items(): 
                last_prob = trading_pair_probs.tail(1).signal_prob[0]
                if last_prob > max_prob:
                    max_prob = last_prob
                    max_trading_pair = trading_pair
        
        # max proba identified and > threshold for the trading pair
        if (max_trading_pair != '') and (max_prob > self.thresholds[max_trading_pair]):
            cash_amount_to_use = self.trading_api.get_available_amount_crypto(self.cash_asset) * self.param_bet_size
            what_to_buy[self.trading_pairs[max_trading_pair]] = cash_amount_to_use
            
        return what_to_buy
    
    # Change bet size regarding proba ?
    def __buy(self, key, trading_pair, amount):       
        id_order = self.trading_api.create_order(trading_pair.base_asset, trading_pair.quote_asset, ORDER_BUY, amount, key)
        order = self.trading_api.get_order(id_order)
        
        # TODO : System that ensure that order is executed, may modify order, etc.
        
        # trace
        if self.trace:
            print('[BUY] ORDER PLACED (' + str(key) + '): ' + str(order.quantity_base) + ' ' + order.base_asset + ' for '
                   + str(order.quantity_quote) + '$ (close_price = ' + str(round(order.price, 2))
                   + '$ / fees = ' + str(round(order.fees_quote_asset, 2)) + ')')
        
        self.signals_x_buy.append(key)
        self.signals_y_buy.append(order.price)        
        
    def __what_to_sell(self, current_date, signals):
        what_to_sell = {}
        for trading_pair, value in self.trading_pairs.items():            
            if signals[trading_pair].signal_prob.max() < self.thresholds[trading_pair]:
                # if crypto currently in portfolio
                crypto_amount = self.trading_api.get_available_amount_crypto(value.base_asset)
                if crypto_amount > 0:
                    what_to_sell[value] = crypto_amount
        return what_to_sell
        
    def __sell(self, key, trading_pair, crypto_amount): # from_crypto, to_crypto
        id_order = self.trading_api.create_order(trading_pair.base_asset, trading_pair.quote_asset, ORDER_SELL, crypto_amount, key)
        order = self.trading_api.get_order(id_order)
        
        if self.trace:
            print('[SELL] ORDER PLACED (' + str(key) + '): ' + str(order.quantity_base) + ' ' + order.base_asset + ' for '
                   + str(order.quantity_quote) + '$ (close_price = ' + str(round(order.price, 2)) 
                   + '$ / fees = ' + str(round(order.fees_quote_asset, 2)) + ')')

        self.x_sell.append(key)
        self.y_sell.append(order.price)
        
        self.amount_x.append(key)
        self.amount_y.append(self.trading_api.get_available_amount_crypto(trading_pair.quote_asset))
        
    def do_sell_all(self, key):
        for trading_pair, value in self.trading_pairs.items():
            amount_available = self.trading_api.get_available_amount_crypto(value.base_asset)
            if amount_available > 0:
                self.__sell(key, value, amount_available)
    
    # check & perform actions that need to be done (buy / sell) at a specific date
    def do_update(self, key, signals):
        # sell
        for trading_pair, amount in self.__what_to_sell(key, signals).items():
            self.__sell(key, trading_pair, amount)            
        # buy
        for trading_pair, amount in self.__what_to_buy(key, signals).items():
            self.__buy(key, trading_pair, amount)
            
    def get_available_amount_crypto(self, symbol):
        return self.trading_api.get_available_amount_crypto(symbol)
    
    def get_all_fees_paid(self):
        fees = 0
        for key, order in self.trading_api.get_orders().items():
            fees = fees + order.fees_quote_asset
        return fees
    
    def get_nb_transactions_done(self):
        return len(self.trading_api.get_orders())
    
    def get_signals(self):
        return self.signals_x_buy, self.signals_y_buy, self.x_sell, self.y_sell, self.amount_x, self.amount_y

# REmplacer from crypto - to crypto par trading pair avec side
class Backtesting_strategy:    
    def __init__(self, model, model_term, init_date, end_date, X_tests, close_price, target, thresholds, trading_pairs, cash_asset, trace=True):
        self.param_init_amount_cash = 1000.0 # $
        self.param_fees = 0.001 # 0.1%
        self.param_bet_size = 1.00 #%
        self.param_min_bet_size = 100.0 # $
        self.signals = {}
        
        self.model = model
        self.model_term = model_term
        self.init_date = init_date
        self.end_date = end_date - timedelta(hours=1) # to avoir getting a price unknown at the end of simulation
        self.X_tests = X_tests
        self.close_price = close_price
        self.target = target
        self.trading_pairs = trading_pairs
        self.cash_asset = cash_asset        
        self.trace = trace
        
#         self.__add_missing_prices()
        
        #self.market_price_begin = close_price[trading_pair][0] # !!!
        #self.market_price_end = close_price[trading_pair][len(close_price[trading_pair]) - 1] # !!
        
        self.__calcul_signals()
        
        # set init positions
        init_positions = {self.cash_asset: self.param_init_amount_cash}
        for key, value in trading_pairs.items():
            init_positions[value.base_asset] = 0.0
        
        # trading API (fake one for simulation)           
        trading_api = Trading_api_fake()
        trading_api.init_from_backtesting_strategy(init_positions, self.param_fees, self.close_price)
        
        # trading module
        self.trading_module = Trading_module(trading_api, self.param_bet_size, self.param_min_bet_size, trading_pairs, self.cash_asset, thresholds, trace)
    
#     # add missing last price needed for simulation
#     def __add_missing_prices(self):
#         for trading_pair, value in self.trading_pairs.items():
#             df = self.close_price[trading_pair]            
#             last_price = df.xs(df.tail(1).index[0])
#             last_date = df.tail(1).index[0]
#             new_date = last_date + timedelta(hours=1)
#             s = pd.Series(last_price)
#             indexes = df.index
#             indexes = indexes.append(pd.DatetimeIndex([new_date]))
#             df = df.append(s, ignore_index=True)
#             df.index = indexes
#             self.close_price[trading_pair] = df
    
    def __calcul_signals(self):
        for trading_pair, value in self.trading_pairs.items():
            predicted_proba = self.model.predict_proba(self.X_tests[trading_pair])
            probs = predicted_proba[:, 1]
            df_probs = pd.DataFrame(probs)
            df_probs.index = self.X_tests[trading_pair].index
            df_probs.columns = ['signal_prob']
            self.signals[trading_pair] = df_probs            
    
    def override_signals(self, signals):
        self.signals = signals
        
    def get_signals(self):
        return self.trading_module.get_signals()
    
    def get_signals_for_date(self, current_date):
        signals_for_date = {}        
        date_before = current_date - timedelta(hours=self.model_term)
        for trading_pair, value in trading_pairs.items():
            signals_for_date[trading_pair] = self.signals[trading_pair].truncate(before=date_before, after=current_date)
        return signals_for_date
    
    def do_backtest(self):
        current_date = self.init_date
        while current_date < self.end_date:
            signals = self.get_signals_for_date(current_date)
            self.trading_module.do_update(current_date, signals)
            current_date = current_date + timedelta(hours=1)
        self.trading_module.do_sell_all(current_date)
        
        # Results
        simulation_time = round((self.end_date - self.init_date).days) # Avec les dates !
        final_amount = round(self.trading_module.get_available_amount_crypto(self.cash_asset), 2)
        all_fees_paid = round(self.trading_module.get_all_fees_paid(), 2)
        pct_change_portfolio = round((final_amount - self.param_init_amount_cash) / self.param_init_amount_cash * 100, 2)
        #pct_change_market = round((self.market_price_end - self.market_price_begin) / self.market_price_begin * 100, 2)
        pct_change_market = 0
        nb_trades = self.trading_module.get_nb_transactions_done()
        
        if self.trace:
            print('\n')
            print('Simulation time: ' + str(simulation_time) + ' days')
            print('Start amount: ' + str(self.param_init_amount_cash) + '$')
            print('Final amount: ' + str(final_amount) + '$')
            print('Number of transactions: ' + str(nb_trades))
            print('Fees: ' + str(all_fees_paid) + '$')
            print('Pourcentage change portfolio: ' + str(pct_change_portfolio) + '%')
            #print('Pourcentage change market: ' + str(pct_change_market) + '%')
        
        return simulation_time, final_amount, all_fees_paid, pct_change_portfolio, pct_change_market, nb_trades
    
    def show_graphs(self):
        # here to avoid problem when started in batch mode
        from matplotlib import pyplot as plt
        signals_x, signals_y, x_sell, y_sell, amount_x, amount_y = self.get_signals()

        plt.figure(figsize=(20,10))
        plt.plot(close_price)
        plt.plot(signals_x, signals_y, '^', markersize=10, color='g')
        plt.plot(x_sell, y_sell, 'v', markersize=10, color='r')
        plt.xlabel('h + n hours')
        plt.ylabel('close_price')
        plt.show()

        plt.figure(figsize=(20,10))
        plt.plot(amount_x, amount_y, color='y')
        plt.xlabel('h + n hours')
        plt.ylabel('portfolio_value')
        plt.show()

In [103]:
base_asset_1 = 'ETH'
quote_asset_1 = 'USDT'
trading_pair_1 = base_asset_1 + quote_asset_1
pair_1 = Trading_pair(trading_pair_1, base_asset_1, quote_asset_1)

base_asset_2 = 'XRP'
quote_asset_2 = 'USDT'
trading_pair_2 = base_asset_2 + quote_asset_2
pair_2 = Trading_pair(trading_pair_2, base_asset_2, quote_asset_2)

trading_pairs = {trading_pair_1: pair_1, trading_pair_2: pair_2}
thresholds = {trading_pair_1: threshold, trading_pair_2: threshold}

init_date = y_test_1.index.min()
end_date = y_test_1.index.max()

backtest = Backtesting_strategy(model_1, 24, init_date, end_date, {trading_pair_1: X_test_1, trading_pair_2: X_test_2}, {trading_pair_1: X_test_close_price_1, trading_pair_2: X_test_close_price_2}, close_price_increase_targeted, thresholds, trading_pairs, quote_asset_1)
# backtest.override_signals(y_test)
backtest.do_backtest()
# backtest.show_graphs()

[BUY] ORDER PLACED (2018-09-12 02:00:00+00:00): 3878.105590062112 XRP for 1000.0$ (close_price = 0.26$ / fees = 1.0)
[SELL] ORDER PLACED (2018-09-13 09:00:00+00:00): 3878.105590062112 XRP for 1054.1772985248447$ (close_price = 0.27$ / fees = 1.06)
[BUY] ORDER PLACED (2018-09-18 07:00:00+00:00): 3873.2001516230966 XRP for 1054.1772985248447$ (close_price = 0.27$ / fees = 1.05)
[SELL] ORDER PLACED (2018-09-24 17:00:00+00:00): 3873.2001516230966 XRP for 2015.9193417166377$ (close_price = 0.52$ / fees = 2.02)
[BUY] ORDER PLACED (2018-09-26 13:00:00+00:00): 3645.074067646916 XRP for 2015.9193417166377$ (close_price = 0.55$ / fees = 2.02)
[SELL] ORDER PLACED (2018-09-28 01:00:00+00:00): 3645.074067646916 XRP for 1964.186799136658$ (close_price = 0.54$ / fees = 1.97)
[BUY] ORDER PLACED (2018-09-28 05:00:00+00:00): 3635.0919087393877 XRP for 1964.186799136658$ (close_price = 0.54$ / fees = 1.96)
[SELL] ORDER PLACED (2018-10-02 01:00:00+00:00): 3635.0919087393877 XRP for 2070.656676956836$ (clo

(88, 2126.24, 44.11, 112.62, 0, 20)

In [102]:
import import_ipynb
import algocryptos_preprocessing as alg_preproc
from xgboost import XGBClassifier

dict_hours_labels  = {24:'1d'}
predict_only_one_crypto = True
y_to_be_considered = 'y_+1d_classif'
close_price_increase_targeted = +5
threshold = 0.7
id_cryptocompare_1 = "7605"
id_cryptocompare_2 = "5031"

dict_df = load_obj('dict_df_2018_11_24_top7')

def train_model(X_train, y_train):
    scale_pos_weight = y_train.value_counts()[False] / y_train.value_counts()[True]
    xgbc = XGBClassifier(random_state=0, scale_pos_weight=scale_pos_weight).fit(X_train, y_train)
    return xgbc

X_train_1, X_test_1, y_train_1, y_test_1, X_train_close_price_1, X_test_close_price_1 = alg_preproc.get_preprocessed_data(dict_df, dict_hours_labels, 
                                                                     close_price_increase_targeted, 
                                                                     True,
                                                                     do_scale=True, 
                                                                     do_pca=True,
                                                                     id_cryptocompare=id_cryptocompare_1)

X_train_2, X_test_2, y_train_2, y_test_2, X_train_close_price_2, X_test_close_price_2 = alg_preproc.get_preprocessed_data(dict_df, dict_hours_labels, 
                                                                     close_price_increase_targeted, 
                                                                     True,
                                                                     do_scale=True, 
                                                                     do_pca=True,
                                                                     id_cryptocompare=id_cryptocompare_2)


model_1 = train_model(X_train_1, y_train_1[y_to_be_considered])
model_2 = train_model(X_train_2, y_train_2[y_to_be_considered])
# TODO : Pourquoi X_train1 et X_train 2 sont différents ? 
# pd.DataFrame(X_train_1).describe() vs pd.DataFrame(X_train_2).describe()